In [ ]:
import yaml
import os
import csv
import math
import matplotlib.pyplot as plt
from collections import deque


In [ ]:
def read_last_xy_from_files(folder):
    x_list = list()
    y_list = list()
    
    # loop over all files in folder
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            # open csv file
            with open(folder + filename, 'r') as f:
                try:
                    # get last values
                    lastrow = deque(csv.reader(f), 1)[0]
                    x = float(lastrow[1])         
                    y = float(lastrow[2])  
                    
                    # store in list
                    x_list.append(-x) 
                    y_list.append(-y)

                except IndexError:  # empty file
                    lastrow = None
    return x_list, y_list

In [ ]:
# CW
err_x_CW, err_y_CW = read_last_xy_from_files("./data_front_022/CW/")

# CCW
err_x_CCW, err_y_CCW = read_last_xy_from_files("./data_front_022/CCW/")

# CW COG
x_cg_CW = sum(err_x_CW)/len(err_x_CW)
y_cg_CW = sum(err_y_CW)/len(err_y_CW)

# CCW COG
x_cg_CCW = sum(err_x_CCW)/len(err_x_CCW)
y_cg_CCW = sum(err_y_CCW)/len(err_y_CCW)

In [ ]:
# length of square
L = 13.03

# used trackwidth
b = 0.22

# calculate E_b (flipped sign because of different coordinate system)
alpha_1 = (x_cg_CW + x_cg_CCW)/(-4.0*L)
alpha_2 = (y_cg_CW - y_cg_CCW)/(-4.0*L)
alpha =  (alpha_1 + alpha_2)/2.0
E_b = (math.pi/2)/(math.pi/2 - alpha) 

# calculate E_d (flipped sign because of different coordinate system)
beta_1 = (x_cg_CW - x_cg_CCW)/(-4.0*L)
beta_2 = (y_cg_CW + y_cg_CCW)/(-4.0*L)
beta = (beta_1 + beta_2)/2.0
R = (L/2.0)/math.sin(beta/2.0)
E_d = (R + b/2.0)/(R - b/2.0)

# print
print("Error E_d: ", E_d)
print("Error E_b: ", E_b)
print("Actual track width b_actual: ", E_b * b)

In [ ]:
# corrected CW
err_x_CW_corr, err_y_CW_corr = read_last_xy_from_files("./data_result/CW/")

# corrected CCW
err_x_CCW_corr, err_y_CCW_corr = read_last_xy_from_files("./data_result/CCW/")

# CW COG corrected
x_cg_CW_corr = sum(err_x_CW_corr)/len(err_x_CW_corr)
y_cg_CW_corr = sum(err_y_CW_corr)/len(err_y_CW_corr)

# CCW COG corrected
x_cg_CCW_corr = sum(err_x_CCW_corr)/len(err_x_CCW_corr)
y_cg_CCW_corr = sum(err_y_CCW_corr)/len(err_y_CCW_corr)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()

# negate elements
err_x_CCW = [-x for x in err_x_CCW]
err_y_CCW = [-x for x in err_y_CCW]
err_x_CW = [-x for x in err_x_CW]
err_y_CW = [-x for x in err_y_CW]
x_cg_CW = - x_cg_CW
x_cg_CCW = - x_cg_CCW
y_cg_CW = - y_cg_CW
y_cg_CCW = - y_cg_CCW

# intermediate results
ax.scatter(err_x_CCW, err_y_CCW, color="green", label="CCW")
ax.scatter(err_x_CW, err_y_CW, color="blue", label="CW")
ax.scatter([x_cg_CW, x_cg_CCW], [y_cg_CW, y_cg_CCW], color="red", marker="+", label="COG")

# negate elements
err_x_CCW_corr = [-x for x in err_x_CCW_corr]
err_y_CCW_corr = [-x for x in err_y_CCW_corr]
err_x_CW_corr = [-x for x in err_x_CW_corr]
err_y_CW_corr = [-x for x in err_y_CW_corr]
x_cg_CW_corr = - x_cg_CW_corr
x_cg_CCW_corr = - x_cg_CCW_corr
y_cg_CW_corr = - y_cg_CW_corr
y_cg_CCW_corr = - y_cg_CCW_corr


# corrected results
ax.scatter(err_x_CCW_corr, err_y_CCW_corr, color="goldenrod", marker="s", label="CCW corrected")
ax.scatter(err_x_CW_corr, err_y_CW_corr, color="sienna", marker="s", label="CW corrected")
ax.scatter([x_cg_CW_corr, x_cg_CCW_corr], [y_cg_CW_corr, y_cg_CCW_corr], color="red", marker="x", label="COG corrected")


# format everything
ax.set_xlabel('x_err in m')
ax.set_ylabel('y_err in m')

# set the x-spine (see below for more info on `set_position`)
ax.spines['left'].set_position('zero')

# turn off the right spine/ticks
ax.spines['right'].set_color('none')
ax.yaxis.tick_left()

# set the y-spine
ax.spines['bottom'].set_position('zero')

# turn off the top spine/ticks
ax.spines['top'].set_color('none')
ax.xaxis.tick_bottom()

# make axis equal
ax.axis('equal')

# legend
fig.legend()

fig.savefig("result.svg", format="svg")

plt.show()